In [1]:
import pyrealsense2 as rs
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.animation as animation
import time
%matplotlib inline

In [2]:
def load_stream_frames(n_frames, resolution=(640, 480)):
    depth_frames = []
    
    w, h = resolution
    
    pipeline = rs.pipeline()
    config = rs.config()
    config.enable_stream(rs.stream.depth, w, h, rs.format.z16,  30)

    # Start streaming
    pipeline.start(config)
    
    try:
        for i in range(n_frames):

            # Wait for a coherent pair of frames: depth and color
            frames = pipeline.wait_for_frames()
            depth_frame = frames.get_depth_frame()
            if not depth_frame:
                continue
                
            depth_frames.append(depth_frame)
            
    finally:
        # Stop streaming
        pipeline.stop()
    return depth_frames

In [3]:
def _mean_time_in_apply_filter(filter_, seq):
    times = np.zeros(len(seq))
    for i, frame in enumerate(seq):
        start_time = time.time()
        filter_.process(frame)
        end_time = time.time()
        times[i] = end_time - start_time
    return times.mean() 
        
def calculate_times(seq, filters):
    times = np.zeros(len(filters), dtype=float)
    
    for i, filter_ in enumerate(filters):
        times[i] = _mean_time_in_apply_filter(filter_, seq)
    
    return times

---

In [4]:
# DATA WITH DIFERENTS RESOLUTIONS

depth_low_resolution =  load_stream_frames(15,  (640, 480))
depth_mid_resolution =  load_stream_frames(15,  (848, 480))
depth_high_resolution = load_stream_frames(15, (1280, 720))

In [5]:
# FILTERS 

dec_filter =  rs.decimation_filter ()  # Decimation - reduces depth frame density
disp_filter = rs.disparity_transform() # Disparity
spat_filter = rs.spatial_filter()      # Spatial - edge-preserving spatial smoothing
temp_filter = rs.temporal_filter()     # Temporal - reduces temporal noise

filters = [dec_filter, disp_filter, spat_filter, temp_filter]

In [6]:
# time in ms.
t_low =  calculate_times(depth_low_resolution, filters)  
t_mid =  calculate_times(depth_mid_resolution, filters)  
t_high = calculate_times(depth_high_resolution, filters) 

In [7]:
pd.DataFrame(np.vstack((t_low, t_mid, t_high)) /1e-3 ,
             index=['Low resolution', 'Mid resolution', 'High resolution'], 
             columns=['Decimation', 'Disparity', 'Spatial', 'Temporal'])

,Decimation,Disparity,Spatial,Temporal
Low resolution,0.759125,0.012334,6.909243,0.761302
Mid resolution,0.821495,0.008392,8.894253,0.916958
High resolution,1.636855,0.008170,18.153254,2.083588


*Time in ms*